# bootstrapping interactive notebook content for _computational thermodynamics of materials_

In [1]:
    import docx, itertools, pandas, tempfile, shutil, typing, asyncio, subprocess, functools, unittest, operator, re
    from IPython.display import Markdown, display; from toolz.curried import *
    PANDOC, PANDOC_TO, PANDOC_FROM = shutil.which("pandoc"), "gfm", "docx"
    MAIN, FILE = __name__ == "__main__", "__file__" in locals()
    INTERACTIVE = MAIN and not FILE
    singleton = functools.lru_cache(1)
    from pathlib import Path
    from utils import apply
    HERE = Path(__file__).parent if FILE else Path(subprocess.check_output(["pwd"]).decode().strip())
    PSU = HERE.parent / "psu410" / "src" / "psu410"

    import nbformat.v4 as v4, anyio, json

## loading the original textbook and data

`get_docx` loads in the original document using `python-docx` data structures.

loads the docx into a dataframe and extracts features for organizing the contents into cells.

In [2]:
    def get_docx_features():
        # start with an index of files, this will work for multiple documents but will be ugly
        p = (
            pandas.Index(["2015-07-15-Textbook-Cambridge-U-P.docx"], name="path")
            .to_series().apply(docx.Document).methodcaller("iter_inner_content")
            .apply(enumerate).apply(list).explode()
            .apply(list).aseries().rename(columns={0: "original", 1: "p"})
            .set_index("original", append=True)
        )
        # extract the ALL style information from the style node
        p = p.assign(**p.p.attrgetter("style.element.attrib").apply(dict).aseries())
        # remove the url prefixes on the style namepsaces
        DOCX_PRE  = "{http://schemas.openxmlformats.org/wordprocessingml/2006/main}"
        p.columns = p.columns.str.removeprefix(DOCX_PRE)

        # extract features from the style ids that map paragraphs to packages, documents, cells
        p = p.assign(
            package=p.styleId.eq("Heading1").cumsum().rename("package"),
            h=(h := p.styleId.str.extract("^Heading([0-9])")[0].dropna().astype(int).reindex_like(p)),
            doc=(hs := p.styleId.str.match("^Heading[1-2]")).cumsum(),
            cell_id=(~h.isna()).astype(int).cumsum()
        ).reset_index()
        return p

    INTERACTIVE and (p := get_docx_features()).T

,0,1,2,3,4,5,6,7,8,9,...,3278,3279,3280,3281,3282,3283,3284,3285,3286,3287
path,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,...,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx,2015-07-15-Textbook-Cambridge-U-P.docx
original,0,1,2,3,4,5,6,7,8,9,...,3278,3279,3280,3281,3282,3283,3284,3285,3286,3287
p,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...,<docx.text.paragraph.Paragraph object at 0x7fe...
type,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,...,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph,paragraph
default,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
styleId,Normal,Normal,TOC1,TOC2,TOC2,TOC2,TOC2,TOC1,TOC2,TOC2,...,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal,Normal
customStyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
package,0,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,10,10
h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
doc,0,0,0,0,0,0,0,0,0,0,...,48,48,48,48,48,48,48,48,48,48


`get_headings` maps the indexes to file names

In [3]:
    def get_headings(p):
        """map package, doc, cell indices to file names"""
        headings = p.dropna(subset="h").set_index(["package", "doc", "cell_id"])
        headings = headings.assign(title=headings.p.attrgetter("text"))
        headings = headings.assign(
            package_slug=headings.title[headings.h.eq(1)].apply(slugify).reindex_like(headings).ffill(),
            document_slug=headings.title[headings.h.le(2)].apply(slugify).reindex_like(headings).ffill(),
        )
        headings.loc[headings[headings.package_slug.eq(headings.document_slug)].index, "document_slug"] = "index"
        headings = headings.assign(
            target=PSU / headings.package_slug / (
                headings.document_slug.add(".ipynb")
            ))
        return headings

    def slugify(s): 
        """convert a string into a slug"""
        return pipe(s, str.lower, partial(re.sub, "\s", "_"), partial(re.sub, "[^\.a-z\_]", ""))

    INTERACTIVE and (headings := get_headings(p))

    def contents_as_tree(contents):
        """`contents_as_tree` shows the prospective file layout of the contents."""
        import rich.tree
        tree = rich.tree.Tree("psu410")
        for parent, files in contents.target.drop_duplicates().pipe(
            lambda s: s.groupby(s.apply(operator.attrgetter("parent")))
        ):
            tree.add(t := rich.tree.Tree(parent.name))
            pipe(files, map(compose_left(operator.attrgetter("name"), t.add)), list)
        return tree

    INTERACTIVE and (tree := contents_as_tree(headings))

psu410
├── applications_to_chemical_reactions
│   ├── index.ipynb
│   ├── internal_process_and_differential_and_integrated_driving_forces.ipynb
│   ├── ellingham_diagram_and_buffered_systems.ipynb
│   ├── trends_of_entropies_of_reactions.ipynb
│   └── maximum_reaction_rate_and_chemical_transport_reactions_.ipynb
├── applications_to_electrochemical_systems
│   ├── index.ipynb
│   ├── electrolyte_reactions_and_electrochemical_reactions.ipynb
│   ├── concentrations_activities_and_reference_states_of_electrolyte_species.ipynb
│   ├── electrochemical_cells_and_half_cell_potentials.ipynb
│   ├── aqueous_solution_and_pourbaix_diagram.ipynb
│   └── application_examples.ipynb
├── calpahd_modeling_of_thermodynamics
│   ├── index.ipynb
│   ├── importance_of_lattice_stability.ipynb
│   ├── modeling_of_pure_elements.ipynb
│   ├── modeling_of_stoichiometric_phases.ipynb
│   ├── modeling_of_random_solution_phases.ipynb
│   └── modeling_of_solution_phases_with_longrange_ordering.ipynb
├── critical_phenomena_thermal_expansion_and_materials_genome
│   ├── index.ipynb
│   ├── mms_model_applied_to_thermal_expansion.ipynb
│   ├── application_to_cerium.ipynb
│   ├── application_to_fept.ipynb
│   └── concept_of_materials_genome.ipynb
├── experimental_data_for_thermodynamic_modeling
│   ├── index.ipynb
│   ├── phase_equilibrium_data_from_experiments.ipynb
│   └── thermodynamic_data_from_experiments.ipynb
├── firstprinciples_calculations_and_theory
│   ├── index.ipynb
│   ├── nickel_as_the_prototype.ipynb
│   ├── firstprinciples_formulation_of_thermodynamics.ipynb
│   ├── quantum_theory_for_the_motion_of_electrons.ipynb
│   ├── lattice_dynamics.ipynb
│   └── firstprinciples_approaches_to_disordered_alloys.ipynb
├── gibbs_energy_function
│   ├── index.ipynb
│   ├── phases_with_fixed_compositions.ipynb
│   ├── phases_with_variable_compositions_random_solutions.ipynb
│   ├── phases_with_variable_compositions_solutions_with_ordering.ipynb
│   ├── polymer_solutions_and_polymer_blends.ipynb
│   └── elastic_magnetic_and_electric_contributions_to_free_energy.ipynb
├── laws_of_thermodynamics
│   ├── index.ipynb
│   ├── first_and_second_laws_of_thermodynamics.ipynb
│   ├── combined_law_of_thermodynamics_and_equilibrium_conditions.ipynb
│   ├── stability_at_equilibrium_and_property_anomaly.ipynb
│   └── gibbsduhem_equation.ipynb
├── phase_equilibria_in_heterogeneous_systems
│   ├── index.ipynb
│   ├── general_condition_of_equilibrium.ipynb
│   ├── gibbs_phase_rule.ipynb
│   ├── potential_phase_diagrams.ipynb
│   └── molar_phase_diagrams.ipynb
└── references
    └── index.ipynb

`get_documents` creates smaller docx articles from the original bulk and convert them to markdown

In [4]:
    async def get_documents(p):
        documents = p.groupby(["package", "doc", "cell_id"]).p.agg(get_new_docx).to_frame("docx")
        # shell out to pandoc to get the markdown form
        documents = documents.assign(md=await documents.docx.apply(get_docx_markdown).gather())
        return documents

    def get_new_docx(elements: list["docx.Type"]) -> "docx.document.Document":
        "creates a new `docx` data structure using only the specified elements."
        try: return (new := docx.Document())
        finally: new._body._element.extend(x._element for x in elements)    
            

    async def get_docx_markdown(document) -> str:
        """save docx to disc and return a stringified markdown version. this is async because it shells out everytime"""
        document.save((tmp := tempfile.NamedTemporaryFile(suffix=".docx")).name)
        return subprocess.check_output([PANDOC, "--to", PANDOC_TO, "--from", PANDOC_FROM, tmp.name]).decode()

    INTERACTIVE and (documents := await get_documents(p)).T

package                                                 0   \
doc                                                     0    
cell_id                                                 0    
docx     <docx.document.Document object at 0x7fef24b1cf50>   
md       Computational Thermodynamics of Materials\n\nZ...   

package                                                 1   \
doc                                                     1    
cell_id                                                 1    
docx     <docx.document.Document object at 0x7fef2535f1d0>   
md                              # Laws of thermodynamics\n   

package                                                     \
doc                                                     2    
cell_id                                                 2    
docx     <docx.document.Document object at 0x7fef256fc210>   
md       ## First and second laws of thermodynamics\n\n...   

package                                                     \
doc                                                     3    
cell_id                                                 3    
docx     <docx.document.Document object at 0x7fef2535f090>   
md       ## Combined law of thermodynamics and equilibr...   

package                                                     \
doc                                                     4    
cell_id                                                 4    
docx     <docx.document.Document object at 0x7fef248cf710>   
md       ## Stability at equilibrium and property anoma...   

package                                                     \
doc                                                     5    
cell_id                                                 5    
docx     <docx.document.Document object at 0x7fef248e18d0>   
md       ## Gibbs-Duhem equation\n\nIn experiments, it ...   

package                                                 2   \
doc                                                     6    
cell_id                                                 6    
docx     <docx.document.Document object at 0x7fef248e3290>   
md       # Gibbs energy function\n\nAs shown in through...   

package                                                     \
doc                                                     7    
cell_id                                                 7    
docx     <docx.document.Document object at 0x7fef248e3710>   
md       ## Phases with fixed compositions\n\nThe homog...   

package                                                     \
doc                                                     8    
cell_id                                                 8    
docx     <docx.document.Document object at 0x7fef248fcc10>   
md       ## Phases with variable compositions: Random s...   

package                                                     ...  \
doc                                                         ...   
cell_id                                                 9   ...   
docx     <docx.document.Document object at 0x7fef24953f10>  ...   
md       ### Random solutions\n\nThe ideal Gibbs energy...  ...   

package                                                 8   \
doc                                                     42   
cell_id                                                 87   
docx     <docx.document.Document object at 0x7fef24fef710>   
md       ### Galvanic protection\n\nA galvanic reaction...   

package                                                     \
doc                                                          
cell_id                                                 88   
docx     <docx.document.Document object at 0x7fef2501e610>   
md       ### Fuel cells\n\nFuel cells are devices to co...   

package                                                     \
doc                                                          
cell_id                                                 89   
docx     <docx.document.Document object at 0x7fef25097a50

`get_notebooks` synthesizes the blocks of markdown into notebook files

In [5]:
    def get_notebooks(documents, headings):
        notebooks = documents.join(headings.target).groupby(["target"]).apply(get_new_notebook).to_frame("nb")
        return notebooks
        
    def get_new_notebook(df):
        """create a new notebook format from markdown paragraphs"""
        return v4.new_notebook(cells=[v4.new_markdown_cell(x.splitlines(True)) for x in df.sort_index().md])

    INTERACTIVE and (notebooks := get_notebooks(documents, headings)).T

target,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_chemical_reactions/ellingham_diagram_and_buffered_systems.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_chemical_reactions/index.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_chemical_reactions/internal_process_and_differential_and_integrated_driving_forces.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_chemical_reactions/maximum_reaction_rate_and_chemical_transport_reactions_.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_chemical_reactions/trends_of_entropies_of_reactions.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_electrochemical_systems/application_examples.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_electrochemical_systems/aqueous_solution_and_pourbaix_diagram.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_electrochemical_systems/concentrations_activities_and_reference_states_of_electrolyte_species.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_electrochemical_systems/electrochemical_cells_and_half_cell_potentials.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/applications_to_electrochemical_systems/electrolyte_reactions_and_electrochemical_reactions.ipynb,...,/home/tbone/Documents/nobook/src/psu410/src/psu410/laws_of_thermodynamics/first_and_second_laws_of_thermodynamics.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/laws_of_thermodynamics/gibbsduhem_equation.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/laws_of_thermodynamics/index.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/laws_of_thermodynamics/stability_at_equilibrium_and_property_anomaly.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/phase_equilibria_in_heterogeneous_systems/general_condition_of_equilibrium.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/phase_equilibria_in_heterogeneous_systems/gibbs_phase_rule.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/phase_equilibria_in_heterogeneous_systems/index.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/phase_equilibria_in_heterogeneous_systems/molar_phase_diagrams.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/phase_equilibria_in_heterogeneous_systems/potential_phase_diagrams.ipynb,/home/tbone/Documents/nobook/src/psu410/src/psu410/references/index.ipynb
nb,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...",...,"{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata...","{'nbformat': 4, 'nbformat_minor': 5, 'metadata..."


## testing

In [6]:
    class Tests(unittest.TestCase):
        
        @classmethod
        def arun(cls, co):
            """an async runner that patches nested event loops when working interactively."""
            try: return asyncio.get_running_loop().run_until_complete(co)
            except RuntimeError: return __import__("nest_asyncio").apply() or cls.arun(co)
        
        def setUp(self):
            self.paragraphs = get_docx_features()
            self.headings = get_headings(self.paragraphs)
            self.documents = self.arun(get_documents(self.paragraphs))
            self.notebooks = get_notebooks(self.documents, self.headings)
            INTERACTIVE and globals().update(self=self)
            
        def test_number_of_sections(self):
            """the table of contents, 9 primary chapters, and the references are captured"""
            # all of the main headings are captured as Heading1s
            assert self.paragraphs.styleId.eq("Heading1").sum() == 10
            assert len(self.documents.index.get_level_values("package").unique()) == 11
            
    INTERACTIVE and (test_results := unittest.main(argv=["discover"], verbosity=2, exit=False))

test_number_of_sections (__main__.Tests.test_number_of_sections)
the table of contents, 9 primary chapters, and the references are captured ... ok

----------------------------------------------------------------------
Ran 1 test in 14.030s

OK


### write the notebooks

In [7]:
    async def write_json(target, data):
        target = anyio.Path(target)
        await target.parent.mkdir(exist_ok=True, parents=True)
        return await target.write_text(json.dumps(data, indent=2))
    
    async def write_nb(notebooks):
        return await asyncio.gather(*(
            write_json(target, nb) for target, nb in notebooks.nb.items()
        ))

    10 and INTERACTIVE and str(await write_nb(notebooks))

'[10326, 1193, 6567, 10446, 3959, 22812, 15674, 4257, 16935, 5052, 1649, 7739, 3254, 3906, 3562, 11518, 2040, 14044, 8518, 4168, 1465, 6998, 1668, 15971, 8319, 14994, 12878, 3663, 18091, 19250, 14173, 8221, 11916, 18271, 34412, 26361, 4538, 12929, 7327, 4717, 230, 12381, 7680, 3372, 249, 40796, 31853, 14905]'

### show an interactive sample of the transformed markdown

In [8]:
    INTERACTIVE and display(*documents.md[documents.md.str.contains("\$")].iloc[:10].apply(Markdown))

## First and second laws of thermodynamics

A system typically consists of many chemical components. The first law
of thermodynamics states that the exchanges of heat, work, and
individual components with its surroundings must obey the law of
conversation of energy. In the domain of materials science and
engineering, the energy of interest is at the atomic and molecular
levels. The energy at the higher and lower levels such as nuclear energy
and kinetic and potential energies of a rigid body are usually excluded
from the discussion of thermodynamics of materials.

Let us consider a system receiving an amount of heat, *dQ*, and an
amount of work, *dW*, and an amount of each independent component *i*,
*dN<sub>i</sub>*, from the surroundings. Such a system is called an open
system in contrast to a closed system when *dN<sub>i</sub>*=0 for all
components, i.e. no exchange of mass between the system and the
surrounding. Other types of systems commonly defined in thermodynamics
include adiabatic systems without exchange of heat, i.e. *dQ*=0, and
isolated systems without exchange of any, i.e. *dQ*= *dW*=
*dN<sub>i</sub>*=0.

The corresponding change of energy in the system, i.e. the internal
energy change, *dU*, is formulated in terms of the first law of
thermodynamics as follows,

Eq. ‑ $dU = dQ + dW + \sum_{}^{}{H_{i}dN_{i}}$

where $H_{i}$ is the unit energy of component *i* in the surroundings,
and the summation is for all components in the system, which can be
controlled independently from the surroundings, i.e. the independent
components of the system.

It is self-evident that the left-hand side of refers to the change
inside the system, while its right-hand side is for the contributions
from the surroundings to the system. In principle, no matter how the
heat and mass are added, or how the work is done to the system, as far
as their summation is the same, the change of the internal energy will
be the same based on the first law of thermodynamics, indicating that
the system reaches the same state for a closed system. The internal
energy is thus a state function in a close system as it does not depend
on how the state is reached.

On the other hand, for the purpose of easy mathematical treatment, a
reversible process can be considered for a closed system, in which the
initial state of the system can be restored reversibly without any
changes left to the surroundings. Therefore, the heat transferred and
the work done to the system are identical to the heat and work lost by
the surroundings and vice versa. The classic example of reversible
processes is the Carnot’s cycle, which is shown in . It consists of four
reversible processes for a closed system. The four reversible processes
are compression at constant temperature *T<sub>1</sub>* (isothermal,),
compression without heat exchange (adiabatic) ending at *T<sub>2</sub>*,
isothermal expansion at *T<sub>2</sub>*, and adiabatic expansion ending
at *T<sub>1</sub>*.

Figure ‑: Schematics of the Carnot’s cycle

The Carnot’s cycle involves a simple type of mechanical work, either
hydrostatic expansion or compression, with the work that the surrounding
does to the system represented by

Eq. ‑ $dW = - PdV$

with *P* being the external pressure that the surrounding exerts on the
system and *V* the volume of the system. It is now necessary to
differentiate the external and internal variables for further discussion
with the former representing variables in the surroundings and the
latter representing variables in the system. For the isothermal
processes in the Carnot’s cycle, the entropy change of the system, *dS*,
can be defined as the heat exchange divided by temperature

Eq. ‑ $dS = \frac{dQ}{T}$

In addition to processes involving heat, work, and mass exchanges
between the system and the surroundings, there could be internal
processes taking place inside the system. As the system cannot do work
to itself, the criterion for whether an internal process can occur
spontaneously must be related to the heat exchange, which is related to
the entropy change as shown by .

It is a known fact that heat can spontaneously transfer from a higher
temperature (*T<sub>2</sub>*) region to a lower temperature
(*T<sub>1</sub>*) region inside a system if the heat conduction is
allowed, and this process is irreversible because heat cannot be
conducted from a low temperature region to a high temperature region by
itself. indicates that for the same amount of heat change, the entropy
change at *T<sub>1</sub>* is higher than that at *T<sub>2</sub>*, and
the heat conduction thus results in a positive entropy change in the
system, i.e.

Eq. ‑
$\mathrm{\Delta}S = - \frac{dQ}{T_{2}} + \frac{dQ}{T_{1}} = \frac{dQ}{{T_{2}T}_{1}}\left( {T_{2} - T}_{1} \right) > 0$

Consequently, the second law of thermodynamics is obtained, which states
that for an internal process to take place spontaneously or
irreversibly, this internal process (*ip*) must have a positive entropy
production, which can be written in a differential form as follows

Eq. ‑ $d_{ip}S > 0$

From the definition of entropy change shown by , the amount of heat
produced by this irreversible internal process can be calculated as
follows

Eq. ‑ $d_{ip}Q = Td_{ip}S$

Let us represent this internal process by *dξ* and define the driving
force for this internal process by *D*. The work done by this internal
process is thus *Ddξ*, which is released as heat, i.e.

Eq. ‑ $Dd\xi = d_{ip}Q = Td_{ip}S$

An irreversible process thus must have a positive driving force in order
for it to take place spontaneously.


## Combined law of thermodynamics and equilibrium conditions

For a system with an irreversible internal process taking place, the
entropy change in the system thus consists of three parts: the heat
exchange with the surrounding defined by , the entropy production due to
the internal process represented by , and the entropy of mass exchange
with the surrounding. The total entropy change of the system can thus be
written as follows

Eq. ‑ $dS = \frac{dQ}{T} + d_{ip}S + \sum_{}^{}{S_{i}dN_{i}}$

where $S_{i}$ is the unit entropy of component *i* in the surroundings,
often called partial entropy of component *i*, and will be further
discussed in Chapter

Combining and and re-arranging, one obtains

Eq. ‑ $dQ = TdS - Dd\xi - \sum_{}^{}{TS_{i}dN_{i}}$

Inserting and into yields the combined law of thermodynamics from the
first and second laws of thermodynamics,

Eq. ‑
$dU = TdS - PdV + \sum_{}^{}\left( H_{i} - TS_{i} \right){dN}_{i} - Dd\xi$

The internal energy of the system is thus a function of *S*, *V*,
*N<sub>i</sub>* and *ξ* of the system, which are called natural
variables of the internal energy, i.e. *U*(*S*,*V*,*N<sub>i</sub>*,*ξ*).
The other variables are dependent variables and can be represented by
partial derivatives of the internal energy with respect to their
respective natural variables with other natural variables kept constant
as shown below

Eq. ‑
$T = \left( \frac{\partial U}{\partial S} \right)_{V,\ N_{i},\ \xi}$

Eq. ‑
$- P = \left( \frac{\partial U}{\partial V} \right)_{S,\ N_{i},\ \xi}$

Eq. ‑
$\mu_{i} = H_{i} - TS_{i} = \left( \frac{\partial U}{\partial N_{i}} \right)_{S,\ V,N_{j \neq i},\ \xi} = U_{i}$

Eq. ‑
$- D = \left( \frac{\partial U}{\partial\xi} \right)_{S,\ V,N_{i}\ }$

In , a new variable, $\mu_{i}$, is introduced. It is called chemical
potential and defined as the internal energy change with respect of the
addition of the component *i* when the entropy, volume and the amount of
other components of the system are kept constant. It may be worth
pointing out that for a system at equilibrium, i.e. $d_{ip}S = 0$, and
with constant entropy, $dS = 0$, if it exchanges mass with the
surroundings, $dN_{i} \neq 0$, the system must also exchange heat with
the surroundings at the same time in order to keep the entropy invariant
as demonstrated by .

The pairs of the natural variables and their corresponding partial
derivatives are called conjugate variables, i.e. *S* and *T*, *V* and
*–P*, *N<sub>i</sub>* and $\mu_{i}$, and *ξ* and *–D*. There are minus
sign in front of *P* and *D* as the increase of volume and the progress
of the internal process decrease the internal energy of the system. The
importance of this conjugate relation will be evident when various forms
of combined thermodynamic laws and various types of phase diagrams are
introduced in the book.

The last pair of conjugate variables, *ξ* and *–D*, is worthy of further
discussion. Based on the second law of thermodynamics, i.e. , no
internal processes take place spontaneously if there is no entropy
productions, i.e. D≤0 or *dξ*=0 and *D*\>0. With D≤0, there is no
driving for any internal processes, and the system is at a full
equilibrium state. The last term in drops off, and *ξ* becomes a
dependent variable of the system and can be calculated from the
equilibrium conditions. With *dξ*=0 and *D*\>0, the system is under a
constrained equilibrium or freezing-in condition when the internal
process is constrained not to take place, and *ξ* remains to be an
independent variable of the system.

These two cases represent the two branches of thermodynamics:
equilibrium, reversible thermodynamics and irreversible thermodynamics.
It is clear from the above discussions that these two branches are
identical if the internal energy is not only a function of *S*, *V*, and
*N<sub>i</sub>* , but also any internal process variable *ξ*. This means
that one should be able to evaluate the internal energy of a system for
any freezing-in equilibrium conditions in addition to the full
equilibrium condition. In the rest of the book, the freezing-in
equilibrium and full equilibrium are not differentiated unless
specified.

As the mechanical work under hydrostatic pressure is very important in
experiments, let us define a new quantity called enthalpy as follows

Eq. ‑ $H = U + PV$

Its differential form can be obtained from as

Eq. ‑ $dH = dU + d(PV) = dQ + VdP + \sum_{}^{}H_{i}{dN}_{i}$

There are two significant consequences of the above equation. First, for
a close system under constant pressure, i.e. ${dN}_{i} = dP = 0$, one
has $dH = dQ$. This implies that the enthalpy change in a system is
equal to the heat exchange between the system and the surrounding of the
system, which is why enthalpy and heat are often exchangeable in the
literature. Second, for an adiabatic system under constant pressure,
i.e. $dQ = dP = 0$, can be re-arranged to the following equation

Eq. ‑
$H_{i} = \left( \frac{\partial H}{\partial N_{i}} \right)_{N_{j \neq i,\ \ dQ = dP = 0}}$

$H_{i}$ is thus the partial enthalpy of component *i* and will be
further discussed in Chapter . The chemical potential of component *i*
defined in is thus related to the partial enthalpy and partial entropy
of the component.

To further define equilibrium conditions of a system, consider a
homogeneous system in a state of internal equilibrium, i.e. no
spontaneous internal processes are possible with $Dd\xi = 0$, and
becomes

Eq. ‑
$dU = TdS - PdV + \sum_{}^{}\mu_{i}{dN}_{i} = \sum_{}^{}{Y_{i}dX_{i}}$

where *X* represents *S*, *V*, *N<sub>i</sub>*, and *Y* their conjugate
variables *T*, *-P*, $\mu_{i}$. The state of the system with *c*
independent components is completely determined by the *c+2* variables,
i.e. *S*, *V*, and *N<sub>i</sub>* with *i* from 1 to *c*.

To simplify the situation, let us limit the discussion to an isolated
equilibrium system, i.e. $dU = 0$, and conduct a virtual internal
experiment inside the system by moving an infinitesimal amount of
$X_{i}$, ${dX}_{i}$, with other $X_{j}$ kept constant, from one region
of the system to another region of the system as schematically shown in
.

Figure ‑: Virtual experiment for a system at equilibrium

As the system is homogeneous and at equilibrium,
$- dX_{i}^{'} = dX_{i}^{"} = dX_{i}$. The total change of the internal
energy of this internal process is the combination of the changes in the
two regions, i.e.

Eq. ‑
$dU = dU^{'} + dU^{"} = Y_{i}^{'}dX_{i}^{'} + Y_{i}^{"}dX_{i}^{"} = \left( - Y_{i}^{'} + Y_{i}^{"} \right)dX_{i} = 0$

Therefore, $Y_{i}^{'} = Y_{i}^{"}$ for *T*, *-P*, and $\mu_{i}$,
indicating that *T*, *-P*, and $\mu_{i}$ are homogeneous in the system,
respectively, and are thus named as potentials of the system.
Furthermore these potentials are independent of the size of the system
and are often referred to as intensive variables in the literature. On
the other hand, all *X:s*, i.e. *S*, *V*, and *N<sub>i</sub>*, are
proportional to the size of the system and may be normalized with
respect to the size of the system, usually in terms of total moles,

Eq. ‑ $N = \sum_{}^{}N_{i}$

They are thus called molar quantities and often referred to as extensive
variables, and the respective normalized variables are molar entropy,
molar volume, and mole fractions, defined as follows

Eq. ‑ $S_{m} = \frac{S}{N}$

Eq. ‑ $V_{m} = \frac{V}{N}$

Eq. ‑ $x_{i} = \frac{N_{i}}{N}$

Consider a small subsystem in this homogeneous system at equilibrium and
let the subsystem grow in size. The entropy, volume, and mass enclosed
in the subsystem increase as follows

Eq. ‑ ${dS = S}_{m}dN$

Eq. ‑ $dV = V_{m}dN$

Eq. ‑ $dN_{i} = x_{i}dN$

The corresponding change in the internal energy of the subsystem becomes

Eq. ‑
$dU = TdS - PdV + \sum_{}^{}\mu_{i}{dN}_{i} = \left( TS_{m} - PV_{m} + \sum_{}^{}\mu_{i}x_{i} \right)dN = U_{m}dN$

By integration one obtains the integral form of the internal energy as

Eq. ‑
$U = TS - PV + \sum_{}^{}\mu_{i}N_{i} = \left( TS_{m} - PV_{m} + \sum_{}^{}\mu_{i}x_{i} \right)N = U_{m}N$

Similarly, the molar enthalpy can be defined as follows

Eq. ‑
$H = U + PV = U_{m}N + PV_{m}N = \left( U_{m} + PV_{m} \right)N = H_{m}N$

In case a potential is not homogeneous in a system, the system will not
be in a state of equilibrium. Let us consider the same virtual
experiment as shown in for an isolated system that is not in
equilibrium, i.e. by moving an infinitesimal amount of $X_{i}$,
${dX}_{i}$, with other $X_{j}$ kept constant, from one region of the
system to another region of the system with the two regions having
different potentials. The total internal energy change equals to zero as
the virtual experiment has $dU = 0$. Similarly, each region can be
considered to be homogeneous by itself, and one has
$- dX_{i}^{'} = dX_{i}^{"} = dX_{i}$. The total internal energy change
in the system is thus the sum of these two regions plus the entropy
production due to the internal process with $d\xi = dX_{i}$, i.e.

Eq. ‑
$dU = dU^{'} + dU^{"} + Dd\xi = Y_{i}^{'}dX_{i}^{'} + Y_{i}^{"}dX_{i}^{"} + Dd\xi = \left( - Y_{i}^{'} + Y_{i}^{"} \right)dX_{i} + Dd\xi = 0$

Consequently, one obtains the following

*Eq. 1‑31* $D = Y_{i}^{'} - Y_{i}^{"}$

The driving force thus represents the difference of the potential at the
two regions, and the internal process is to eliminate inhomogeneity of
the potential with the heat transfer from high temperature regions to
low temperature regions, volume shrink of low pressure regions (high
$–P$) and volume expansion of high pressure regions (low $–P$), and
transport of components from high chemical potential regions to low
chemical potential regions.


## Stability at equilibrium and property anomaly

As shown by , potentials are homogenous for a homogeneous system in a
state of internal equilibrium. To study the stability of the equilibrium
state, one considers the entropy production due to a fluctuation of a
molar quantity as an internal process. Based on the second law of
thermodynamics, the driving force, as the first derivative of the
entropy production with respect to the internal process, is zero for
such a fluctuation at equilibrium, i.e. *D*=0, and the entropy of
production thus depends on the second derivative. It can be written as
follows

Eq. ‑
$Td_{ip}S = \frac{\partial_{ip}S}{\partial\xi}d\xi + \frac{1}{2}{\frac{\partial_{ip}^{2}S}{\partial\xi^{2}}(d\xi)}^{2} = Dd\xi - \frac{1}{2}D_{2}(d\xi)^{2}$

with $D_{2} = - \frac{\partial_{ip}^{2}S}{\partial\xi^{2}}$. When
$\frac{\partial_{ip}^{2}S}{\partial\xi^{2}} < 0$ or $D_{2} > 0$ along
with $D = 0$, the fluctuation does not produce positive entropy of
production and can thus not develop further. The equilibrium state of
the system is therefore stable against the fluctuation. On the other
hand, when $\frac{\partial_{ip}^{2}S}{\partial\xi^{2}} > 0$ or
$D_{2} < 0$ along with $D = 0$, the fluctuation creates positive entropy
of production and can continue to grow. The equilibrium state of the
system is therefore unstable against the fluctuation. In connection with
, one can realize that for a system at stable equilibrium without heat
and mass exchanges with the surroundings, its entropy is at its maximum,
and there are no other internal processes, which could produce any more
entropy. This is schematically shown in .

Figure ‑: Schematic diagram showing maximum entropy

Using , , and , the combined law of thermodynamics can be written as

*Eq. 1‑33*
$dU = \sum_{}^{}{Y_{i}dX_{i}} - Dd\xi + \frac{1}{2}D_{2}(d\xi)^{2}$

Let us carry out the same virtual internal experiment shown in Chapter ,
i.e. moving an infinitesimal amount of *X<sub>i</sub>* in a homogenous
system with other $X_{j}$ kept constant in an isolated system, i.e.
$dU = 0$ and $D = 0$. The internal energy change due to this internal
process is

Eq. ‑
$dU = \frac{1}{2}D_{2}\left\{ \left( dX_{i}^{'} \right)^{2} + \left( dX_{i}^{"} \right)^{2} \right\}$

For a homogeneous system in a state of stable equilibrium with
$\left( dX_{i}^{'} \right)^{2} = \left( dX_{i}^{"} \right)^{2} = \left( dX_{i} \right)^{2}$,
this internal process must result in an increase of internal energy,
$dU > 0$, and thus gives

Eq. ‑
$D_{2} = 2\left( \frac{\partial^{2}U}{\partial\left( X_{i} \right)^{2}} \right)_{X_{j}} = 2\left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{X_{j}} > 0$

shows that for a system to be stable, any pair of conjugate variables
must change in the same direction when other independent molar
quantities are kept constant. For the conjugate variables discussed so
far, it means that for a stable system, the addition of entropy
increases temperature with $\frac{\partial T}{\partial S}$\>0, the
volume decreases with pressure or increases with the negative of
pressure with $\frac{\partial( - P)}{\partial V} > 0$, and the chemical
potential of a component increases with its amount, i.e.
$\frac{\partial\mu_{i}}{\partial N_{i}} > 0$, where the derivatives are
taken with all other molar quantities kept constant. The limit of
stability is reached when becomes zero, i.e.

Eq. ‑
$D_{2} = 2\left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{X_{j}} = 0$

shows schematically the energy as a function of configurations including
three states: unstable, stable, and metastable. Both the stable and
metastable states have positive curvatures due to $D_{2} > 0$, while the
unstable state has a negative curvature due to $D_{2} < 0$. There is an
inflection point of $D_{2} = 0$ for a state between a stable or
metastable state with $D_{2} > 0$ and an unstable state with
$D_{2} < 0$. These two inflection points, called spinodal, represent the
limit of stability. The states between the two inflection points are
unstable, and other states are either stable or metastable. The two
inflection points can move apart from or close to each other depending
on the change of external conditions, i.e. the natural variables. One
extreme situation is when these two inflection points merge into one
point, and the instability occurs only at this particular point. It is
evident that all three states, stable, metastable, and unstable, also
merge into one point. This point is called critical or consolute point,
beyond which the instability no longer exists.

Figure ‑: Schematic diagram showing the stable and unstable equilibrium
states

To mathematically define the consolute point, the third derivative needs
to be added to because both $D$ and $D_{2}$ vanish at this point, i.e.

Eq. ‑
$Td_{ip}S = \frac{\partial_{ip}S}{\partial\xi}d\xi + \frac{1}{2}{\frac{\partial_{ip}^{2}S}{\partial\xi^{2}}(d\xi)}^{2} + \frac{1}{6}{\frac{\partial_{ip}^{3}S}{\partial\xi^{3}}(d\xi)}^{3} = Dd\xi - \frac{1}{2}D_{2}(d\xi)^{2} + \frac{1}{6}{D_{3}(d\xi)}^{3}$

Eq. ‑
$dU = \sum_{}^{}{Y_{i}dX_{i}} - Dd\xi + \frac{1}{2}D_{2}(d\xi)^{2} - \frac{1}{6}{D_{3}(d\xi)}^{3}$

At the consolute point, the third derivative also becomes zero, i.e.

Eq. ‑ $D_{3} = \frac{\partial_{ip}^{3}S}{\partial\xi^{3}}^{3} = 0$

Let us further discuss the properties of the system in relation to the
critical point. By taking the inverse of the equation of the limit of
stability, , one obtains

Eq. ‑
$\left( \frac{\partial X_{i}}{\partial Y_{i}} \right)_{X_{j}} = + \ \infty$

i.e. all $X_{i}$ quantities diverge at the critical point. Therefore,
when a system approaches the critical point from its stable region, the
change of a molar quantity with respect to its conjugate potential
varies dramatically and becomes infinite at the critical point,
resulting in property anomalies in the system. In the unstable region,
the system would thus separate into stable subsystems and becomes
heterogeneous, and $X_{i}$:s change discontinuously between subsystems.
While in the stable region, the change of a molar quantity with respect
to its conjugate potential decreases as the system moves away from the
critical point and remains positive due to the stability criteria
denoted by .

However, it is not clear how a molar quantity changes with respect to a
non-conjugate potential at the critical point. From the Maxwell
relation, one has

Eq. ‑
$\left( \frac{\partial Y_{i}}{\partial X_{j}} \right)_{X_{k \neq j}} = \frac{\partial^{2}U}{\partial X_{i}\partial X_{j}} = \left( \frac{\partial Y_{j}}{\partial X_{i}} \right)_{X_{k \neq i}}$

Eq. ‑
$\left( \frac{\partial X_{j}}{\partial Y_{i}} \right)_{X_{k \neq j}} = \left( \frac{\partial X_{i}}{\partial Y_{j}} \right)_{X_{k \neq i}}$

Since all $X_{i}$:s diverge at the critical point, both derivatives in
should also go to infinite at the critical point. To investigate their
signs, let us carry out a virtual experiment similar to the one in
deriving the stability condition ( and ). In this case, two internal
processes are needed for moving two molar quantities simultaneously in
an isolated system, i.e.

*Eq. 1‑43*
$dU = - D_{\xi_{1}}d\xi_{1} - D_{\xi_{2}}d\xi_{2} + D_{\xi_{1}\xi_{2}}d\xi_{1}d\xi_{2} + \frac{1}{2}D_{2\xi_{1}}\left( d\xi_{1} \right)^{2} + \frac{1}{2}D_{2\xi_{2}}\left( d\xi_{2} \right)^{2}$

Based on the above discussions, in a stable system at equilibrium with
$D_{\xi_{1}} = D_{\xi_{2}} = 0$, $D_{2\xi_{1}} > 0$ and
$D_{2\xi_{2}} > 0$, the sign of $D_{\xi_{1}\xi_{2}}$ cannot be
unambiguously determined in keeping the change of internal energy
positive, i.e. $dU > 0$. This indicates that the quantities in can be
either positive or negative in the stable region and become zero at the
critical point. By the same token, the quantities in can be either
positive or negative and become positive or negative infinite at the
critical point.

A profound conclusion from this analysis is that in a stable system even
though a molar quantity always changes in the same direction as its
conjugate potential, the same molar quantity may change in the opposite
direction of a non-conjugate potential, resulting in additional
anomalies represented by Eq. 1‑40. One example of is the thermal
expansion in a closed system, i.e. $dN_{i} = 0$, as follows

Eq. ‑
$\left( \frac{\partial V}{\partial T} \right)_{S} = \left( \frac{\partial S}{\partial( - P)} \right)_{V}$

The left-hand side of can be understood as follows: with the increase of
temperature, the system regulates its pressure in order to keep the
entropy from increasing, which results in the volume change of the
system. The behavior of the system depends on whether the pressure
decreases or increases in order to maintain the entropy of the system
constant. If the pressure decreases to maintain the entropy of the
system constant, the volume would increase with the increase of
temperature, i.e. the left-hand side of the equation has a positive
sign, which is also shown by the right-hand side of the equation as the
changes of $S$ and $–P$ have the same sign. That the volume increases
with temperature is the normal scenario. On the other hand, if the
pressure increases to maintain the entropy of the system constant, the
volume would decrease with the increase of temperature, resulting in a
negative sign for the left-hand side of the equation. This decrease of
volume with the increase of temperature is usually considered to be
anomalous, originated from the increase of entropy by the decrease of
$–P$ or the increase of pressure. More discussions on entropy will
follow in Chapter 5.2.5 and Chapter 9.


## Gibbs-Duhem equation

In experiments, it is difficult to control *S* and *V* of a system in
comparison with their conjugate variables *T* and *-P*. It is thus
desirable to construct new functions to represent the system with *T*
and *-P* as natural variables of the functions. One of them is enthalpy
defined in , and other two can be defined as follows

Eq. ‑ $F = U - TS$

Eq. ‑ $G = U - TS + PV = \sum_{}^{}\mu_{i}N_{i} = H - TS = F + PV$

with *F* and *G* called Helmholtz energy and Gibbs energy, respectively.
The middle part of is obtained using $U$ from . The corresponding
combined law of thermodynamics in terms of *H*, *F*, and *G* can be
obtained through the Legendre transformation of as

Eq. ‑ $dH = TdS - Vd( - P) + \sum_{}^{}\mu_{i}{dN}_{i} - Dd\xi$

Eq. ‑ $dF = - SdT - PdV + \sum_{}^{}\mu_{i}{dN}_{i} - Dd\xi$

Eq. ‑ $dG = - SdT - Vd( - P) + \sum_{}^{}\mu_{i}{dN}_{i} - Dd\xi$

The independent variables in each of the above form are regarded as the
natural variables to the corresponding function. The integral forms of
all the functions can thus be written as the following with their
natural variables listed in the parenthesis

Eq. ‑ $U = U\left( S,V,N_{i},\xi \right)$

Eq. ‑ $H = H\left( S, - P,N_{i},\xi \right)$

Eq. ‑ $F = F\left( T,V,N_{i},\xi \right)$

Eq. ‑ $G = G\left( T, - P,N_{i},\xi \right)$

By differentiating , one obtains

Eq. ‑
$dG = \sum_{}^{}\mu_{i}{dN}_{i} + \sum_{}^{}{N_{i}d\mu}_{i} = - SdT - Vd( - P) + \sum_{}^{}\mu_{i}{dN}_{i} - Dd\xi$

For a system at equilibrium, $Dd\xi = 0$, re-arranging gives the
Gibbs-Duhem equation

Eq. ‑ $0 = - SdT - Vd( - P) - \sum_{}^{}{N_{i}d\mu}_{i}$

This equation indicates that for a homogeneous system with *c*
independent components at equilibrium, there is a direct relation among
all the *c+2* potentials, and they are $c$ chemical potentials
($\mu_{i}$), temperature, and pressure. Consequently, only *c+1*
potentials can change independently, and the remaining potential is
dependent on the other potentials. As discussed in connection with ,
there are $c + 2$ independent variables for an equilibrium system with
*c* independent components, where all of them are molar quantities.

With the relationships between potentials and molar quantities defined
by to , one can switch between potentials and molar quantities as
natural variables of the system. For example, one can define a new free
energy function when the chemical potential of one component is
controlled from the surroundings instead of its content and obtain the
following combined first and second law of thermodynamics

Eq. ‑ $\Phi = G - \mu_{1}N_{1} = \sum_{i = 2}^{c}{\mu_{i}N_{i}}$

Eq. ‑
$d\Phi = - SdT - Vd( - P) - N_{1}{d\mu}_{1} + \sum_{i = 2}^{c}{\mu_{i}{dN}_{i}} - Dd\xi$

However, even though the $c + 2$ molar quantities are independent of
each other, indicates that not all the $c + 2$ potentials are
independent, i.e., if chemical potentials of all components are changed
to natural variables, one would obtain . Therefore, among the *c+2*
independent variables used to define the system, the maximum number of
independent potential is *c+1*, and at least one of the *c+2*
independent variables must be a molar quantity. This variable is usually
chosen to be the size of the system or the major element in the system.
The Gibbs-Duhem equation is used to derive Gibbs phase rule in
heterogeneous systems, which is discussed in Chapter of the book.


# Gibbs energy function

As shown in through , all functions have $N_{i}$ and $\xi$ as natural
variables while they differ in other two natural variables. In typical
materials-related experiments, temperature and pressure are the two
variables controlled. They are also the natural variables of Gibbs
energy. Consequently, Gibbs energy is the most widely used function in
thermodynamics of materials science. The rest of this book focuses on
Gibbs energy for this reason. In this chapter, the mathematical formulas
for Gibbs energy of phases with fixed and variable compositions are
discussed which are needed for quantitative calculations of Gibbs energy
under given values of its natural variables.

From , the molar Gibbs energy can be defined as

Eq. ‑
$G_{m}\left( T,P,x_{i},\xi \right) = \frac{G}{N} = \sum_{}^{}\mu_{i}x_{i}$

The molar entropy, molar volume, chemical potential, and the driving
force can be obtained from as

Eq. ‑
$S_{m} = \frac{S}{N} = - \frac{1}{N}\left( \frac{\partial G}{\partial T} \right)_{P,\ N_{i},\ \xi} = {- \left( \frac{\partial G_{m}}{\partial T} \right)}_{P,\ x_{i},\ \xi}$

Eq. ‑
$V_{m} = \frac{V}{N} = \frac{1}{N}\left( \frac{\partial G}{\partial P} \right)_{T,\ N_{i},\ \xi} = \left( \frac{\partial G_{m}}{\partial P} \right)_{T,\ x_{i},\ \xi}$

Eq. ‑
$\mu_{i} = \left( \frac{\partial G}{\partial N_{i}} \right)_{T,P,N_{j \neq i},\ \xi}$

Eq. ‑
$- D = \left( \frac{\partial G}{\partial\xi} \right)_{T,P,N_{i}\ }$

Based on , the molar enthalpy is written as

Eq. ‑ $H_{m} = G_{m} + TS_{m}$

Other physical properties of the system can also be represented by the
partial derivatives of Gibbs energy such as heat capacity, $C_{P}$,
volume thermal expansivity, $\alpha_{V}$, isothermal compressibility,
$\kappa_{T}$, as follows under constant pressure or temperature

Eq. ‑
$C_{P} = \left( \frac{\partial Q}{\partial T} \right)_{P} = \left( \frac{\partial H}{\partial T} \right)_{P} = T\left( \frac{\partial(G + TS)}{\partial T} \right)_{P} = T\left( \frac{\partial S}{\partial T} \right)_{P} = - T\left( \frac{\partial^{2}G}{\partial T^{2}} \right)_{P}$

Eq. ‑
$\alpha_{V} = \frac{\left( \frac{\partial V}{\partial T} \right)_{P}}{V} = \frac{\left( \frac{\left( \partial G/\partial( - P) \right)_{T}}{\partial T} \right)_{P}}{\left( \partial G/\partial( - P) \right)_{T}} = \frac{\frac{\partial^{2}G}{\partial T\partial( - P)}}{\left( \partial G/\partial( - P) \right)_{T}}$

Eq. ‑
$\kappa_{T} = \frac{\left( \frac{\partial V}{\partial( - P)} \right)_{T}}{V} = \frac{\left( \frac{\left( \partial G/\partial( - P) \right)_{T}}{\partial( - P)} \right)_{T}}{\left( \partial G/\partial( - P) \right)_{T}} = \frac{\frac{\partial^{2}G}{\partial( - P)^{2}}}{\left( \partial G/\partial( - P) \right)_{T}} = \frac{1}{B}$

where the $N_{i}$ and $\xi$ are kept constant for all partial
derivatives, and $B$ is the bulk modulus.

In , $G$ cannot be directly replaced by $G_{m}$ because *N* also depends
on *N<sub>i</sub>*. The thermodynamic quantities under such conditions,
i.e. varying the amount of a component at constant temperature and
pressure, are called partial quantities which are introduced in Eq. 1‑8
for partial entropy and for partial enthalpy. This definition can be
extended to all molar quantities such as partial volume and partial
Gibbs energy. Partial quantities of a molar quantity, $A$, can thus be
defined in general as

Eq. ‑
$A_{i} = \left( \frac{\partial A}{\partial N_{i}} \right)_{T,P,N_{j \neq i},\ \xi}$

The general differential form of a molar quantity for a system at
equilibrium can be represented by its partial quantities as

Eq. ‑
$dA = \left( \frac{\partial A}{\partial T} \right)dT + \left( \frac{\partial A}{\partial P} \right)dP + \sum_{}^{}\left( \frac{\partial A}{\partial N_{i}} \right){dN}_{i}$

where the subscripts representing variables kept constant, i.e. the
remaining natural variables of Gibbs energy not in the denominator, are
omitted for simplicity. This will be done throughout the book unless
specified otherwise.

Using the following relations: $A = NA_{m}$, $N = \sum_{}^{}N_{j}$,
$x_{i} = N_{i}/N$,
$\frac{{\partial x}_{i}}{{\partial N}_{i}} = \left( 1 - x_{i} \right)/N$,
and $\frac{{\partial x}_{k}}{{\partial N}_{i}} = {- x}_{k}/N$, can be
derived as, under constant T and P,

Eq. ‑
$A_{i} = A_{m} + N\sum_{j = 1}^{c}{\frac{\partial A_{m}}{\partial x_{j}}\frac{\partial x_{j}}{\partial N_{i}}} = A_{m} + \frac{\partial A_{m}}{\partial x_{i}} - \sum_{j = 1}^{c}x_{j}\frac{\partial A_{m}}{\partial x_{j}}$

where the summation is for all *c* components and the partial
derivatives are taken with other mole fractions kept constant. However,
mole fractions are not independent, but follow the relation
$\sum_{}^{}x_{i} = 1$. Taking $x_{1} = 1 - \sum_{j = 2}^{c}x_{j}$ as the
dependent mole fraction, can be rewritten as

Eq. ‑
$A_{i} = A_{m} + \left( \frac{\partial A_{m}}{\partial x_{i}} - \frac{\partial A_{m}}{\partial x_{1}} \right) - \sum_{j = 2}^{c}x_{j}\left( \frac{\partial A_{m}}{\partial x_{j}} - \frac{\partial A_{m}}{\partial x_{1}} \right)$

The difference of the partial derivatives in the parenthesis in
represents the partial derivative of $A_{m}$ with respect to the mole
fraction of one component when the first component is selected as the
dependent component. Applying and to Gibbs energy, the partial Gibbs
energy or chemical potential of component $i$ is obtained as

Eq. ‑
$\mu_{i} = G_{i} = G_{m} + \frac{\partial G_{m}}{\partial x_{i}} - \sum_{j = 1}^{c}x_{j}\frac{\partial G_{m}}{\partial x_{j}} = G_{m} + \left( \frac{\partial G_{m}}{\partial x_{i}} - \frac{\partial G_{m}}{\partial x_{1}} \right) - \sum_{j = 2}^{c}x_{j}\left( \frac{\partial G_{m}}{\partial x_{j}} - \frac{\partial G_{m}}{\partial x_{1}} \right)$

The derivatives in the stability equation, , are defined with the molar
quantities kept constant. On the other hand, Gibbs energy has two
potentials, temperature and pressure, as natural variables instead. One
would thus need to compare the stability conditions when a variable kept
fixed is changed from a molar quantity to its conjugate potential. This
can be carried out through the use of Jacobians to change the
independent variables

Eq. ‑
$\frac{\partial\left( Y_{i},Y_{j} \right)}{\partial\left( X_{i},X_{j} \right)} = \left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{Y_{j}}\left( \frac{\partial Y_{j}}{\partial X_{j}} \right)_{X_{i}} = \left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{X_{j}}\left( \frac{\partial Y_{j}}{\partial X_{j}} \right)_{X_{i}} - \left( \frac{\partial Y_{i}}{\partial X_{j}} \right)_{X_{i}}\left( \frac{\partial Y_{j}}{\partial X_{i}} \right)_{X_{j}}$

For a stable system, both
$\left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{X_{j}}$ and
$\left( \frac{\partial Y_{j}}{\partial X_{j}} \right)_{X_{i}}$ are
positive based on . Using the Maxwell relation shown by , one thus
obtains

Eq. ‑
$\left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{X_{j}} - \left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{Y_{j}} = \left( \frac{\partial Y_{i}}{\partial X_{j}} \right)_{X_{i}}\left( \frac{\partial Y_{j}}{\partial X_{i}} \right)_{X_{j}}/\left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{X_{j}} \geq 0$

This means that
$\left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{Y_{j}}$ will go
to zero before
$\left( \frac{\partial Y_{i}}{\partial X_{i}} \right)_{X_{j}}$ does. It
indicates that the stability condition becomes more restrictive when
potentials are kept constant in place of their conjugate molar
quantities. Based on the Gibbs-Duhem equation of , the maximum number of
independent potentials is *c+1*, and the last potential is dependent,
i.e.

Eq. ‑
$\left( \frac{\partial Y_{c + 2}}{\partial X_{c + 2}} \right)_{Y_{j \leq c + 1}} = 0$

Therefore, the limit of stability is determined when the derivative
becomes zero with one molar quantity kept constant, e.g.

Eq. ‑
$\left( \frac{\partial Y_{c + 1}}{\partial X_{c + 1}} \right)_{Y_{j < c + 1},X_{c + 2}} = 0$

This is because this derivative reaches zero faster than any other
derivatives with more molar quantities kept constant. shows that all
molar quantities diverge at the limit of stability. The consolute point
is obtained with $c$ additional conditions as follows based on

Eq. ‑
$\left( \frac{\partial^{2}Y_{i}}{\partial\left( X_{i} \right)^{2}} \right)_{Y_{j \leq c + 1, \neq i},X_{c + 2}} = 0$

Together with , all $c + 1$ independent potentials at the consolute
point can be determined. It is evident that the consolute point is a
zero-dimensional point in a two-dimensional space of independent
potentials in a one-component system. With the addition of a second
component to form a binary system, this consolute point in the
one-component system extends into a one-dimensional line. This line
represents the limit of stability of the binary system, and a consolute
point is located at the end of this line. It is thus evident that in a
system with $c$ independent components, the limit of stability is a
*c-1*-dimensional hypersurface in a space of $c + 1$ independent
potentials, while the consolute point is a zero-dimensional point in all
systems, which may be called the invariant critical point.


## Phases with fixed compositions

The homogeneous system discussed so far means that there is only one
phase in the system, i.e. a single-phase system. A phase with a fixed
composition can be a pure element or a stoichiometric compound. There is
thus only one independent component in the system. A stoichiometric
compound contains more than one element, but the relative amounts of
each element are fixed by the stoichiometry and cannot vary
independently, i.e., $dN_{i} = x_{i}dN$. The combined law of
thermodynamics becomes

Eq. ‑
$dG = - SdT - Vd( - P) + \left( \sum_{}^{}{x_{i}\mu_{i}} \right)dN - Dd\xi = - SdT - Vd( - P) + G_{m}dN - Dd\xi$

$G_{m}$ is the molar Gibbs energy of the stoichiometric compound and can
be regarded as the chemical potential of the stoichiometric
phase,$\ \alpha$,

Eq. ‑ $G_{m} = \mu^{\alpha} = \sum_{}^{}{x_{i}\mu_{i}}$

The chemical potential of individual components in the phase cannot be
defined because the amount of each component cannot be varied
independently. For a stoichiometric phase of $N$ moles of atoms at
equilibrium with $dG$=$Nd\mu^{\alpha} + \mu^{\alpha}dN$, reduces to

Eq. ‑ $\ 0 = - SdT - Vd( - P) - Nd\mu^{\alpha}$

which is the Gibbs-Duhem equation, , applied to a stoichiometric phase.
It can be represented graphically by a surface in a three-dimensional
space composed of $\ \mu^{\alpha}$, *T* and *–P*. The direction of the
surface is represented by the three partial directives between any two
of $\ \mu^{\alpha}$, *T* and *–P* with the third one kept constant, i.e.

Eq. ‑
$\ \left( \frac{\partial\mu^{\alpha}}{\partial T} \right)_{P} = - \frac{S}{N} = - S_{m}$

Eq. ‑
$\ \left( \frac{\partial\mu^{\alpha}}{\partial( - P)} \right)_{T} = - \frac{V}{N} = - V_{m}$

Eq. ‑
$\ \left( \frac{\partial( - P)}{\partial T} \right)_{\mu^{\alpha}} = - \frac{S}{V} = - \frac{S_{m}}{V_{m}}$

Based on Nernst’s heat theorem, the entropy difference between two
crystals approaches zero when the temperature approaches absolute zero.
It is thus a common practice to put $S = 0$ for a crystal at 0 K. This
is usually referred as the third law of thermodynamics. From the
definition of entropy change by , $S$ or $S_{m}$ is always positive at
finite temperatures as the system or the crystal absorbs heat from the
surroundings to increase its temperature. $V$ or $V_{m}$ of a phase is a
well-defined physical quantity, and its absolute value can be given and
is always positive. The above three equations can be written in a
general form as

Eq. ‑
$\ \left( \frac{\partial Y_{i}}{\partial Y_{j}} \right)_{Y_{k}} = - \frac{X_{j}}{X_{i}} < 0$

The surface thus has negative slopes in all its directions. The
curvature of the surface can be derived from

Eq. ‑
$\ \left( \frac{\partial^{2}Y_{i}}{\partial\left( Y_{j} \right)^{2}} \right)_{Y_{k}} = - \left( \frac{\partial\left( \frac{X_{j}}{X_{i}} \right)}{\partial Y_{j}} \right)_{Y_{k}} = - \frac{1}{X_{i}}\left( \frac{\partial X_{j}}{\partial Y_{j}} \right)_{Y_{k}} + \frac{X_{j}}{\left( X_{i} \right)^{2}}\left( \frac{\partial X_{i}}{\partial Y_{j}} \right)_{Y_{k}} = - \frac{1}{X_{i}}\left\lbrack \left( \frac{\partial X_{j}}{\partial Y_{j}} \right)_{Y_{k}} - \frac{X_{j}}{X_{i}}{\left( \frac{\partial X_{i}}{\partial Y_{i}} \right)\left( \frac{\partial Y_{i}}{\partial Y_{j}} \right)}_{Y_{k}} \right\rbrack = - \frac{1}{X_{i}}\left\lbrack \left( \frac{\partial X_{j}}{\partial Y_{j}} \right)_{Y_{k}} + \left( \frac{X_{j}}{X_{i}} \right)^{2}\left( \frac{\partial X_{i}}{\partial Y_{i}} \right)_{Y_{k}} \right\rbrack < 0$

Both terms inside the last bracket are positive for a system in a state
of stable internal equilibrium, and the surface thus has a negative
curvature and is convex everywhere as shown in .

Figure ‑: Gibbs energy of a one-component phase as a function of
temperature and negative pressure, showing the convex shape

From experimental observations, it is known that
$S_{m}^{vapor} \gg S_{m}^{liquid} > S_{m}^{solid}$. The curves of
$G_{m}$ or $\mu^{\alpha}$ plotted with respect to $T$ at constant $P$
would thus have the most negative slope for a vapour phase followed by
its liquid and solid phases. As an example, shows Gibbs energy of Zn in
its solid, liquid, and vapour forms as a function of $T$ at constant
$P = 1$ atmospheric pressure.

Figure ‑: Molar Gibbs energy of Zn as a function of T at constant P

Similarly it is common that
$V_{m}^{vapor} \gg V_{m}^{liquid} > V_{m}^{solid}$, and the curves of
$G_{m}$ or $\mu^{\alpha}$ plotted with respect to $P$ at constant $T$
would thus have the most positive slope for a vapour phase followed by
its liquid and solid phases, though there are cases that
$V_{m}^{liquid} < V_{m}^{solid}$ such as those of water and ice. As an
example, shows Gibbs energy of Fe in its two solid (fcc and bcc),
liquid, and vapour forms as a function of $P$ at constant $T = 1273K$.

Figure ‑: Molar Gibbs energy of Zn as a function of P at constant T

The quantities measurable by experiments typically include temperature,
pressure, volume, composition, and amount of heat flow in the combined
law of thermodynamics discussed so far. By measuring the heat needed to
increase the temperature of a phase, the heat capacity of the phase is
obtained as shown by Eq. 2‑7. A typical heat capacity curve as a
function of temperature is shown in for fcc-Al, hcp-Mg, and an
intermetallic phase Al<sub>12</sub>Mg<sub>17</sub>.

Figure ‑: Heat capacity of fcc-Al, hcp-Mg, and
Al<sub>12</sub>Mg<sub>17</sub> as a function of temperature

There are various theoretical models for the heat capacity under
constant volume to be discussed in Chapter 5 of this book, which is
defined as

Eq. ‑
$C_{V} = \left( \frac{\partial U}{\partial T} \right)_{V} = T\left( \frac{\partial(F + TS)}{\partial T} \right)_{V} = T\left( \frac{\partial S}{\partial T} \right)_{V} = - T\left( \frac{\partial^{2}F}{\partial T^{2}} \right)_{V}$

To establish the relationship between $C_{P}$ defined by and $C_{V}$,
$U$ needs to be represented as a function of $T$ and $V$ in terms of $G$
and its derivatives with respect to Gibbs energy’s natural variables of
$T$ and $P$. It can be done as follows

Eq. ‑
$dV = \frac{\partial V}{\partial T}dT + \frac{\partial V}{\partial( - P)}d( - P) = - \frac{\partial^{2}G}{\partial T( - P)}dT - \frac{\partial^{2}G}{\partial( - P)^{2}}d( - P)$

Eq. ‑
$dU = \frac{\partial(G + TS - PV)}{\partial T}dT + \frac{\partial(G + TS - PV)}{\partial( - P)}d( - P) = - \left( T\frac{\partial^{2}G}{\partial T^{2}} - P\frac{\partial^{2}G}{\partial T( - P)} \right)dT—\left( T\frac{\partial^{2}G}{\partial T( - P)} + P\frac{\partial^{2}G}{\partial( - P)^{2}} \right)\left( - \frac{1}{\frac{\partial^{2}G}{\partial( - P)^{2}}}dV + \frac{\frac{\partial^{2}G}{\partial T( - P)}}{\frac{\partial^{2}G}{\partial( - P)^{2}}}dT \right) = - \left\lbrack T\frac{\partial^{2}G}{\partial T^{2}} - T\frac{\left( \frac{\partial^{2}G}{\partial T( - P)} \right)^{2}}{\frac{\partial^{2}G}{\partial( - P)^{2}}} \right\rbrack dT + \left( - T\frac{\frac{\partial^{2}G}{\partial T( - P)}}{\frac{\partial^{2}G}{\partial( - P)^{2}}} + P \right)dV$

Eq. ‑
$C_{V} = C_{P} + T\frac{\left( \frac{\partial^{2}G}{\partial T( - P)} \right)^{2}}{\frac{\partial^{2}G}{\partial( - P)^{2}}} = C_{P} - \frac{\alpha_{V}^{2}VT}{\kappa_{T}} = C_{P} - \alpha_{V}^{2}BVT$

where the thermal expansion, $\alpha_{V}$, and the compressibility or
bulk modulus, $\kappa_{T}$ or $B$, are defined by and , respectively.
From the heat capacity, the enthalpy and entropy can be obtained by
integration of at a constant pressure

Eq. ‑
$S = S_{0} + \int_{0}^{T}\frac{C_{P}}{T}dT = S_{0} + \int_{0}^{298.15}\frac{C_{P}}{T}dT + \int_{298.15}^{T}\frac{C_{P}}{T}dT = S_{298.15} + \int_{298.15}^{T}\frac{C_{P}}{T}dT$

Eq. ‑
$H = H_{0} + \int_{0}^{T}C_{P}dT = H_{0} + \int_{0}^{298.15}C_{P}dT + \int_{298.15}^{T}C_{P}dT = H_{298.15} + \int_{298.15}^{T}C_{P}dT$

In the above equations, two temperature ranges of integration are chosen
for practical applications as most processing procedures in the field of
materials science and engineering take place at temperatures above the
room temperature. Based on the third-law of thermodynamics, $S_{0} = 0$,
$S_{298.15}$ can be obtained by integration. On the other hand for
$H_{0} = U_{0} + PV$, one does not know the absolute value of the
internal energy and thus have to select a reference state for $H$. In
principle, the reference state can be arbitrarily chosen. A widely used
reference state in the thermodynamic modeling practice is to set
$H_{298.15}^{SER} = 0$ at ambient pressure for pure elements at their
respective stable structures at room temperature, called stable element
reference (SER) state with

Eq. ‑
$G_{298.15}^{SER} = H_{298.15}^{SER} - TS_{298.15}^{SER} = - TS_{298.15}^{SER}$

It is further noted that after defining $S_{298.15}$ and $H_{298.15}$,
one only needs the heat capacity at higher temperatures. This makes the
mathematical representation of heat capacity simpler due to a relatively
simple temperature dependence of heat capacity at higher temperatures in
comparison with the variation at lower temperatures. One common
expression for heat capacity at high temperatures and ambient pressure
is as follows

Eq. ‑ $C_{P} = c + dT + \frac{e}{T^{2}} + fT^{2}$

where c, d, e, and f are parameters fitted to experimental or theoretic
data and compiled in various handbooks.

The corresponding $S$, $H$, and $G$ are obtained as

Eq. ‑ $S = b^{'} + clnT + dT - \frac{e}{{2T}^{2}} + \frac{f}{2}T^{2}$

Eq. ‑ $H = a + cT + \frac{d}{2}T^{2} - \frac{e}{T} + \frac{f}{3}T^{3}$

Eq. ‑
$G = H - TS = a - bT - cTlnT - \frac{d}{2}T^{2} - \frac{e}{2T} - \frac{f}{6}T^{3}$

with $b = b^{'} - c$. The integration constants $b^{'}$ and $a$ are
evaluated from $S_{298.15}$ and $H_{298.15}$. As an example, the
enthalpy and entropy of Zn in solid (hcp), liquid, and gas forms are
plotted in and , respectively. The distances between any two curves in
and represent the enthalpy or entropy differences between the two
phases. It can be seen that the gas has much higher enthalpy and entropy
than the solid and liquid.

Figure ‑: Enthalpy of Zn as a function of temperature at one atmospheric
pressure

Figure ‑: Entropy of Zn as a function of temperature at one atmospheric
pressure

Similarly, one can add the pressure dependence into the Gibbs energy
function such as

Eq. ‑
$G = a - bT - cTlnT - \frac{d}{2}T^{2} - \frac{e}{2T} - \frac{f}{6}T^{3} + gP + hTP + mP^{2}$

where g, h, and m are parameters fitted to experimental or theoretic
data and compiled in various handbooks.

The expression for $V$ can be derived as

Eq. ‑ $V = g + hT + 2mP$

The Helmholtz energy can be expressed as a function of its natural
variables by solving $P\ $ from

Eq. ‑
$F = G - PV = a - bT - cTlnT - \frac{d}{2}T^{2} - \frac{e}{2T} - \frac{f}{6}T^{3} - \frac{(g + hT - V)^{2}}{4m}$

In the literature there are many models to represent the temperature and
pressure dependences of thermodynamic properties. The Gibbs energy
difference between a stoichiometric compound and the components at their
reference states of which the compound is composed,
${_{\ }^{0}G}_{i}^{ref}$, is termed as Gibbs energy of formation, i.e.

Eq. ‑ $\mathrm{\Delta}_{f}G = G - \sum_{}^{}N_{i}{_{\ }^{0}G}_{i}^{ref}$

with $N_{i}$ being the stoichiometry of the compound. Similarly,
enthalpy of formation, entropy of formation, and heat capacity of
formation with respect to components at their reference states,
$_{\ }^{0}H_{i}^{ref}$, $_{\ }^{0}S_{i}^{ref}$, and
$_{\ }^{0}{C_{P}}_{i}^{ref}$, can be defined as

Eq. ‑ $\mathrm{\Delta}_{f}H = H - \sum_{}^{}{N_{i}_{\ }^{0}H_{i}^{ref}}$

Eq. ‑ $\mathrm{\Delta}_{f}S = S - \sum_{}^{}{N_{i}_{\ }^{0}S_{i}^{ref}}$

Eq. ‑
$\mathrm{\Delta}_{f}C_{P} = C_{P} - \sum_{}^{}{N_{i}_{\ }^{0}{C_{P}}_{i}^{ref}}$

It should be mentioned that one mole of a compound usually refers to one
mole of formula of stoichiometry of the compound. With a formula like
$A_{a}B_{b}C_{c}$, the compound is composed of total $(a + b + c)$ moles
of components. One should thus be very careful when dealing with
numerical values to be sure whether the data is in terms of per mole of
formula or per mole of components. At the same time the reference states
must be clearly defined. When the SER state defined in is selected as
the reference state, the above formation quantities are called standard
formation quantities such as standard enthalpy of formation.

Since there are only two independent potentials in a one-component
system, its limit of stability can be evaluated with one potential kept
constant, i.e. either $T$ or $P$. Consequently, either Helmholtz energy
or enthalpy is to be used in deriving the limit of stability of a
homogeneous system. For the practical usefulness, let us use Helmholtz
energy because its natural variables of $T$ and *V* are measurable
quantities in typical experiments, while one of the natural variables of
enthalpy, entropy, is not. From and , the limit of stability for a
one-component system at constant temperature can be written as

Eq. ‑
$\left( \frac{\partial( - P)}{\partial V} \right)_{T,N} = F_{VV} = \frac{1}{V\kappa_{T}} = \frac{B}{V} = 0$

where the isothermal compressibility and bulk modulus, $\kappa_{T}$ and
$B$, are defined in . The limit of stability is thus determined when the
isothermal compressibility diverges or the bulk modulus becomes zero
because $V$ has finite values at any temperatures. It is evident that
Helmholtz energy must have higher order dependence on volume than in for
a system with instability because $F_{VV}$ from is constant.

From , the consolute point is defined by

Eq. ‑
$F_{VVV} = \left( \frac{\partial^{2}( - P)}{\partial V^{2}} \right)_{T,N} = \frac{\partial\left( \frac{1}{V\kappa_{T}} \right)}{\partial V} = - \frac{1 + \frac{V}{\kappa_{T}}\frac{\partial\kappa_{T}}{\partial V}}{\kappa_{T}V^{2}} = 0$

Since $\kappa_{T}$ becomes infinite at the limit of stability,
$\frac{\partial\kappa_{T}}{\partial V}$ approaches negative infinite
when the critical/consolute point is approached so that
$\frac{V}{\kappa_{T}}\frac{\partial\kappa_{T}}{\partial V} = - 1$ and
$F_{VVV} = 0$.


## Phases with variable compositions: Random solutions

The combined law of thermodynamics and the Gibbs-Duhem equation of a
solution phase with variable compositions are shown by and ,
respectively. A phase can be represented by a *c+1*-dimensional surface
in a *c+2*-dimensional space of potentials based on the Gibbs-Duhem
equation. The directions and curvature of the surface are represented by
the partial derivatives shown by and secondary derivatives shown by ,
both being negative for a stable phase. To develop a mathematical
formula for Gibbs energy of a phase with variable compositions, one can
consider a phase as a mixture of independent components that the phase
is made of. Its Gibbs energy function can be postulated as the summation
of Gibbs energy of the independent components of the same structure of
the solution, $_{\ }^{0}G_{i}$, plus the contribution due to the mixing,
$_{\ }^{mixing}G$ or $_{\ }^{M}G$

Eq. ‑ $G = \sum_{}^{}{N_{i}_{\ }^{0}G_{i}} +_{\ }^{M}G$

Since the system size usually is not important in thermodynamics, its
properties are typically normalized to one mole with its composition
represented by mole fractions of components. The molar Gibbs energy is
obtained as shown below with the molar Gibbs energy of mixing separated
into two parts: ideal Gibbs energy of mixing assuming no chemical
interaction among components, $_{\ }^{ideal}G_{m}$ or $_{\ }^{I}G_{m}$,
and excess Gibbs energy of mixing due to chemical reaction among
components, $_{\ }^{excess}G_{m}$ or $_{\ }^{E}G_{m}$

Eq. ‑
$G_{m} = \sum_{}^{}{x_{i}_{\ }^{0}G_{i}} +_{\ }^{M}G_{m} = \sum_{}^{}{x_{i}_{\ }^{0}G_{i}} +_{\ }^{I}G_{m} +_{\ }^{E}G_{m}$

From , the chemical potential of a component is thus

Eq. ‑
$\mu_{i} =_{\ }^{0}G_{i} +_{\ }^{I}{G_{m} +}_{\ }^{E}G_{m} + \frac{\partial\left(_{\ }^{I}{G_{m} +}_{\ }^{E}G_{m} \right)}{\partial x_{i}} - \sum_{j = 1}^{c}x_{j}\frac{\partial\left(_{\ }^{I}{G_{m} +}_{\ }^{E}G_{m} \right)}{\partial x_{j}}$

One may define the chemical activity of component *i*, $a_{i}^{\ }$, as
follows

Eq. ‑
$RTlna_{i}^{\ } = \mu_{i} -_{\ }^{0}G_{i}^{\ } =_{\ }^{I}{G_{m} +}\frac{\partial_{\ }^{I}G_{m}}{\partial x_{i}} - \sum_{j = 1}^{c}x_{j}\frac{\partial_{\ }^{I}G_{m}}{\partial x_{j}} +_{\ }^{E}G_{m} + \frac{\partial_{\ }^{E}G_{m}}{\partial x_{i}} - \sum_{j = 1}^{c}x_{j}\frac{\partial_{\ }^{E}G_{m}}{\partial x_{j}}$

In this definition, the chemical activity or simply activity is
calculated with respect to the pure elements in the structure of the
solution for practical reasons as one would like to understand the
chemical potential difference of components in the solution and by
itself with the same structure. It should be noted that this reference
state for chemical activity is usually different from the SER reference
state defined in as the solution may have a different structure than
those of pure components in their SER states. On the other hand, the
activity under the SER reference state can be easily obtained by
replacing $_{\ }^{0}G_{i}$ with ${_{\ }^{0}G}_{i}^{SER}$ from . In
principle, one may choose any structure as the reference state for
activity to be useful for practical applications, i.e.

Eq. ‑ $RTlna_{i}^{ref} = \mu_{i} -_{\ }^{0}G_{i}^{ref}$

For example, the activity of a component in a liquid solution is defined
with respect to the pure component in its liquid form from , but can
also be referred to its SER state which is solid using . The following
sections will discuss in more details how components mix when they are
brought together including concepts such as random mixing, short-range
ordering, and long-range ordering.

The limit of stability of a solution with respect to composition
fluctuation under constant *T*, *P*, and *N<sub>1</sub>* can be derived
as follows from and

Eq. ‑
$\left( \frac{\partial\mu_{i}}{\partial N_{i}} \right)_{T,P,N_{j \neq i},i > 1} > \left( \frac{\partial\mu_{i}}{\partial N_{i}} \right)_{T,P,N_{1},{\mu_{2},N}_{j \neq i},i,j > 2}\ldots... > \left( \frac{\partial\mu_{c}}{\partial N_{c}} \right)_{T,P,N_{1},\mu_{2}..\mu_{c - 1}} = 0$

The first term can be derived from as follows

Eq. ‑
$\left( \frac{\partial\mu_{i}}{\partial N_{i}} \right)_{T,P,N_{j \neq i},i > 1} = \sum_{j = 1}^{c}\frac{\partial_{\ }^{2}G_{m}}{\partial x_{i}\partial x_{j}}\frac{\partial x_{j}}{\partial N_{i}} - \sum_{j = 1}^{c}{x_{j}\sum_{k = 1}^{c}{\frac{\partial_{\ }^{2}G_{m}}{\partial x_{j}\partial x_{k}}\frac{\partial x_{k}}{\partial N_{i}}}} = \frac{1}{N}\left( \frac{\partial_{\ }^{2}G_{m}}{\partial x_{i}^{2}} - \sum_{j = 1}^{c}{x_{j}\frac{\partial_{\ }^{2}G_{m}}{\partial x_{j}^{2}}} - \sum_{j = 1}^{c}{x_{j}\frac{\partial_{\ }^{2}G_{m}}{\partial x_{i}\partial x_{j}}} + \sum_{j = 1}^{c}{\sum_{k = 1}^{c}{{x_{j}x}_{k}\frac{\partial_{\ }^{2}G_{m}}{\partial x_{j}\partial x_{k}}}} \right)$

Denoting
$G_{ij} = \left( \frac{\partial\mu_{i}}{\partial N_{j}} \right)_{T,P,N_{k \neq j}}$and
using to change the variables kept constant from molar quantities to
potentials one-by-one (see \[1\]), the limit of stability can be further
obtained as

Eq. ‑
$\left( \frac{\partial\mu_{c}}{\partial N_{c}} \right)_{T,P,N_{1},\mu_{2}..\mu_{c - 1}} = \frac{\det\left( G_{ij}:2 \leq i,j \leq c \right)}{\det\left( G_{ij}:2 \leq i,j \leq c - 1 \right)} = 0$

where $\det$ stands for determinant. indicates that
$\det\left( G_{ij}:2 \leq i,j \leq c \right) = 0$ at the limit of
stability. Considering $x_{1} = 1 - \sum_{j \neq 1}^{}x_{j}$, let us
introduce

Eq. ‑
$g_{i} = \mu_{i} - \mu_{1} = \left( \frac{\partial G_{m}}{\partial x_{i}} \right)_{x_{k \neq i}} - \left( \frac{\partial G_{m}}{\partial x_{1}} \right)_{x_{k \neq 1}}$

and

Eq. ‑
$g_{ij} = \frac{\partial g_{i}}{\partial x_{j}} = \frac{\partial\left( \mu_{i} - \mu_{1} \right)}{\partial x_{j}} = \frac{\partial^{2}G_{m}}{\partial x_{i}\partial x_{j}} - \frac{\partial^{2}G_{m}}{\partial x_{1}\partial x_{j}} - \frac{\partial^{2}G_{m}}{\partial x_{i}\partial x_{1}} + \frac{\partial^{2}G_{m}}{\partial\left( x_{1} \right)^{2}}$

The limit of stability can be re-written as

Eq. ‑
$\left( \frac{\partial\left( \mu_{c} - \mu_{1} \right)}{\partial x_{c}} \right)_{T,P,N,\mu_{2} - \mu_{1},\ ...\mu_{c - 1} - \mu_{1}} = \frac{\det\left( g_{ij}:2 \leq i,j \leq c \right)}{\det\left( g_{ij}:2 \leq i,j \leq c - 1 \right)} = 0$

i.e. $\det\left( g_{ij}:2 \leq i,j \leq c \right) = 0$. The consolute
point can be defined following

Eq. ‑
$\left( \frac{\partial^{2}\mu_{c}}{\partial\left( N_{c} \right)^{2}} \right)_{T,P,N_{1},\mu_{2}..\mu_{c - 1}} = \left( \frac{\partial^{2}\left( \mu_{c} - \mu_{1} \right)}{\partial\left( x_{c} \right)^{2}} \right)_{T,P,N,\mu_{2} - \mu_{1},\ ...\mu_{c - 1} - \mu_{1}} = 0$

No closed mathematic form has been published in the literature.


### Random solutions

The ideal Gibbs energy of mixing represents an ideal solution in which
all sites are equivalent and the distributions of components on the
sites are completely random. The number of different configurations to
arrange all components is

Eq. ‑ $w = \frac{N!}{\prod_{}^{}\left( N_{i}! \right)}$

Based on Boltzmann’s relation from statistic thermodynamics when all
configurations have the same probability to be observed, the ideal
configurational molar entropy of mixing due to the distribution is

Eq. ‑
$_{\ }^{ideal}S_{m} =_{\ }^{I}S_{m} = \frac{Rlnw}{N} = R\frac{lnN! - \sum_{}^{}\ln\left( N_{i}! \right)}{N} \cong R\frac{NlnN - \sum_{}^{}{N_{i}l{nN}_{i}}}{N} = - R\sum_{}^{}{x_{i}l{nx}_{i}}$

where $R$ is the gas constant. represents the entropy difference between
the ideal solution and the states when individual components are by
themselves, i.e. the mechanical mixing of components. As $x_{i}$ is
smaller than unity, the entropy production to form an ideal solution
from pure components is thus positive, indicating that it is a
spontaneous process. In such an ideal solution, it is assumed that there
are no interactions between components, and the enthalpy of mixing is
thus zero as the internal energy and the volume of the system do not
change. The ideal Gibbs energy of mixing is written as

Eq. ‑ $_{\ }^{I}G = - T_{\ }^{I}S_{m} = RT\sum_{}^{}{x_{i}l{nx}_{i}}$

The Gibbs energy of real solutions, i.e. , becomes

Eq. ‑
$G_{m} = \sum_{}^{}{x_{i}_{\ }^{0}G_{i}} + RT\sum_{}^{}{x_{i}l{nx}_{i}} +_{\ }^{E}G_{m}$

From , the chemical potential is obtained as

Eq. ‑
$\mu_{i} = G_{i} =_{\ }^{0}G_{i} + RTlnx_{i} +_{\ }^{E}G_{m} + \frac{\partial_{\ }^{E}G_{m}}{\partial x_{i}} - \sum_{j = 1}^{c}x_{j}\frac{\partial_{\ }^{E}G_{m}}{\partial x_{j}}$

From the chemical activity of , the activity coefficient, $\gamma_{i}$,
can be defined as follows

Eq. ‑
$\gamma_{i} = \frac{a_{i}}{x_{i}} = \frac{1}{x_{i}}\exp\frac{G_{i} -_{\ }^{0}G_{i}\ }{RT}$

The solution is an ideal solution with $\gamma_{i} = 1$, and is said to
be positively or negatively deviating from an ideal solution with
$\gamma_{i} > 1$ or $\gamma_{i} < 1$, respectively. The chemical
potential is related to the activity and activity coefficient by the
following equation

Eq. ‑
$\mu_{i} =_{\ }^{0}G_{i} + RTlna_{i} =_{\ }^{0}G_{i} + RTln\gamma_{i}x_{i} =_{\ }^{0}G_{i} + RTlnx_{i} + RTln\gamma_{i}$

Let us further exam in more details in order to better understand the
relation between $G_{m}$ and $\mu_{i}$. The partial derivatives in
represent the directions of molar Gibbs energy in the composition space,
i.e. the tangents of molar Gibbs energy with respect to mole fractions
of independent components. Collectively, they define the
multi-dimensional tangent plane of molar Gibbs energy at the given
composition, $x_{i}^{0}$. The mathematical representation of this
tangent plane, $z_{G_{m}}$, is defined by its directional derivatives
and the distance from the point where the derivatives are taken,

Eq. ‑
$z_{G_{m}} = G_{m}\left( x_{i}^{0} \right) + \sum_{i = 1}^{c}{\left( \frac{\partial G_{m}}{\partial x_{i}} \right)_{x_{i}^{0}}\left( x_{i} - x_{i}^{0} \right)}$

The intercept of this tangent plane at each pure component axis, i.e.
$x_{i} = 1$ and $x_{j \neq i} = 0$, is obtained as

Eq. ‑
$z_{G_{m},x_{i} = 1} = G_{m}\left( x_{i}^{0} \right) + \left( \frac{\partial G_{m}}{\partial x_{i}} \right)_{x_{i}^{\ } = x_{i}^{0}} - \sum_{j = 1}^{c}{x_{j}^{0}\left( \frac{\partial G_{m}}{\partial x_{j}} \right)_{x_{i}^{\ } = x_{i}^{0}}}$

This is identical to at the point $x_{i}^{0}$. It is thus shown that the
chemical potential of a component in a solution is represented by the
intercept of the tangent plane of Gibbs energy of the solution on the
$G_{m}$ axis of the component. The distance between the intercept and
the Gibbs energy of the pure component in the same structure of the
solution is related to the chemical activity of the component as defined
by . On the other hand, it is evident that one can choose any other
structure of the pure element to define the chemical activity to compare
chemical potential of the component as shown by .

The stability of a solution is evaluated following , and the derivatives
of chemical potential with respect to its moles, i.e. the elements in
the determinant, are obtained as follows from and ,

Eq. ‑
$\frac{N}{RT}\frac{\partial\mu_{i}}{\partial N_{i}} = \frac{N}{RT}G_{ii} = \frac{1 - x_{i}}{x_{i}} + \frac{1}{\gamma_{i}}\left( \frac{\partial\gamma_{i}}{\partial x_{i}} - \sum_{j = 1}^{c}{x_{j}\frac{\partial\gamma_{i}}{\partial x_{j}}} \right)$

Eq. ‑
$\frac{N}{RT}\frac{\partial\mu_{i}}{\partial N_{k}} = \frac{N}{RT}G_{ik} = - \frac{x_{k}}{x_{i}} + \frac{1}{\gamma_{i}}\left( 1 - \sum_{j = 1}^{c}{x_{j}\frac{\partial\gamma_{i}}{\partial x_{j}}} \right)$

To further study Gibbs energy of solution phases, let us discuss the
details on the excess Gibbs energy of mixing. At this point, one can
start with lower-order systems with fewer components, i.e. two component
and three-component systems, noting that the Gibbs energy of phases with
one component is already presented in Chapter .


### Binary random solutions

From , the Gibbs-Duhem equation of a binary system consisting of
components $A$ and $B$ is written as

Eq. ‑ $0 = - SdT - Vd( - P) - {N_{A}d\mu}_{A} - {N_{B}d\mu}_{B}$

This equation represents a four-dimensional surface. It is self-evident
that both and hold for stable binary solutions too, i.e. the directions
and the curvature of the surface are all negative. To visualize the
four-dimensional surface in the three-dimensional space, one needs to
fix one of the four potentials. As $T$ and $P$ are the natural variables
of Gibbs energy, they are usually chosen to be kept constant. One can
typically investigate behaviors of systems consisting of condensed
phases by varying the temperature at constant pressure. at constant
pressure thus becomes

Eq. ‑ $0 = - SdT - {N_{A}d\mu}_{A} - {N_{B}d\mu}_{B}$

Similar to and , the property of a phase can be represented by a
two-dimensional surface in the three-dimensional space composed of $T$,
$\mu_{A}$, and $\mu_{B}$ under constant $P$, keeping in mind the
following

Eq. ‑
$G_{m} = x_{A}\mu_{A} + x_{B}\mu_{B} = x_{A}_{\ }^{0}G_{A} + x_{B}_{\ }^{0}G_{B} + RT\left( x_{A}l{nx}_{A} + x_{B}l{nx}_{B} \right) +_{\ }^{E}G_{m}$

Since $_{\ }^{E}G_{m}$ must be zero for pure components $A$ and $B$, it
needs to be in the following form

Eq. ‑ $_{\ }^{E}G_{m} = x_{A}x_{B}L_{AB}$

with $L_{AB}$ being a parameter denoting the interaction between
components $A$ and $B$, called interaction parameter. When $L_{AB} = 0$,
the solution is an ideal solution. When $L_{AB}$ is a non-zero constant
independent of temperature and composition, the solution is called a
regular solution. Its excess entropy and excess enthalpy of mixing are
obtained as

Eq. ‑ $_{\ }^{E}S_{m} = \frac{\partial_{\ }^{E}G_{m}}{\partial T} = 0$

Eq. ‑
$_{\ }^{E}H_{m} =_{\ }^{E}G_{m} - T_{\ }^{E}S_{m} = x_{A}x_{B}L_{AB}$

The chemical potential of component $A$ or $B$ in a binary regular
solution can be derived as

Eq. ‑
$\mu_{i} =_{\ }^{0}G_{i} + RTlnx_{i} + \left( 1 - x_{i} \right)^{2}L_{AB}$

In a dilute solution with $x_{i} \rightarrow 0$, one can have

Eq. ‑
$RTln\gamma_{i} = \left( 1 - x_{i} \right)^{2}L_{AB} \approx L_{AB}$

Eq. ‑ $\gamma_{i} = e^{\frac{L_{AB}}{RT}}$

The activity is thus proportional to its mole fraction, which is called
Henry’s law. By the same token, for the solvent, i.e.
$x_{i} \rightarrow 1$,

Eq. ‑ $RTln\gamma_{i} = \left( 1 - x_{i} \right)^{2}L_{AB} \approx 0$

which gives $\gamma_{i} \approx 1$, and its activity approaches its mole
fraction. This is called Raoult’s law.

The stability of a binary solution is derived from as

Eq. ‑
$\left( \frac{\partial\mu_{A}}{\partial N_{A}} \right)_{T,P,N_{B}} = \left\lbrack \frac{RT}{x_{A}} - 2\left( 1 - x_{A} \right)L_{AB} \right\rbrack\frac{1 - x_{A}}{N}$

Eq. ‑
$\left( \frac{\partial\mu_{B}}{\partial N_{B}} \right)_{T,P,N_{A}} = \left\lbrack \frac{RT}{x_{B}} - 2\left( 1 - x_{B} \right)L_{AB} \right\rbrack\frac{1 - x_{B}}{N}$

It should be noted that the two chemical potentials in a binary system
at constant temperature and pressure are dependent on each other due to
the Gibbs-Duhem equation shown in , i.e.

Eq. ‑ $0 = - {N_{A}d\mu}_{A} - {N_{B}d\mu}_{B}$

and the two chemical potentials depend on each other by the following
relation

Eq. ‑
$\left( \frac{\partial\mu_{A}}{\partial\mu_{B}} \right)_{T,P} = - \frac{N_{B}}{N_{A}}$

Therefore, at the limit of stability, both and go to zero at the same
time, which is obtained when

Eq. ‑ $RT = 2{x_{A}x_{B}L}_{AB}$

As the absolute temperature cannot be negative, has no solution for a
solution phase with $L_{AB} < 0$, i.e. the solution phase is stable with
respect to the composition fluctuation. For a solution with
$L_{AB} > 0$, its limit of stability is represented .

A schematic molar Gibbs energy of a solution with $L_{AB} < 0$ at
constant temperature and pressure is shown in along with the ideal and
excess Gibbs energy of mixing. A tangent line on the molar Gibbs energy
of the solution is drawn, and its two intercepts at $x_{B} = 0$ and
$x_{B} = 1$ give the chemical potentials of components $A$ and $B$,
$\mu_{A}$ and $\mu_{B}$ by , respectively. It is evident that $\mu_{A}$
and $\mu_{B}$ are not independent on each other as they are two points
on the same straight line. This is a graphic representation of the
Gibbs-Duhem equation of . The chemical activity of component $B$ is also
depicted with the reference state being the pure B with the same
structure. As shown in , other structures of pure B can be selected as
the reference states of the chemical activity of component B, resulting
in the different distances to its chemical potential in the solution,
thus different values of its chemical activities. It is clear that this
change of reference state for chemical activity does not affect the
chemical potential of the component in the solution.

Figure ‑: Schematic molar Gibbs energy diagram with $L_{AB} < 0$

When $L_{AB} > 0$, represents a parabola in the $T - x_{i}$
two-dimensional coordinate, symmetric with respect to $x_{A}$ and
$x_{B}$, shown in , i.e. the spinodal of the solution. The consolute
point is obtained by applying to and letting equal to zero at the
consolute point

Eq. ‑
$\left( \frac{\partial^{2}\mu_{A}}{\partial N_{A}^{2}} \right)_{T,P,N_{B}} = \left\lbrack - \frac{RT}{x_{A}^{2}} + 2L_{AB} \right\rbrack\left( \frac{1 - x_{A}}{N} \right)^{2} = 0$

which gives

Eq. ‑ $T_{cons} = 2x_{A}^{2}L_{AB}$

Solving and , one obtains $x_{A} = x_{B} = 0.5$ and

Eq. ‑ $T_{cons} = \frac{L_{AB}}{2R}$

Figure ‑: A Spinodal curve with $L_{AB} > 0$

A schematic molar Gibbs energy diagram at temperatures below the
consolute point is shown in . It can be seen that part of the molar
Gibbs energy has negative curvature, and the solution becomes unstable.
The chemical potential thus does not change monotonically with respect
to composition and its derivative changes sign at the inflexion point.

Figure ‑: Schematic molar Gibbs energy diagram with $L_{AB} > 0$

For more complex solutions, $L_{AB}$ can be a function of temperature,
pressure, and compositions. In principle, the temperature and pressure
dependences can be treated by means of formula similar to . There are
various approaches in the literature to consider the composition
dependence of $L_{AB}$. The empirical Redlich-Kister polynomial stands
out as the one most widely used because it can be extrapolated to
ternary and multi-component systems consistently, which will be
discussed in Chapter .


### Ternary random solutions

From , the Gibbs-Duhem equation of a ternary system consisting of
components $A$, $B$ and $C$ is written as

Eq. ‑
$0 = - SdT - Vd( - P) - {N_{A}d\mu}_{A} - {N_{B}d\mu}_{B} - {N_{C}d\mu}_{C}$

This equation represents a five-dimensional surface. It can be
visualized in a three-dimensional space with two of the five potentials
fixed. Usually $T$ and $P$ are kept constant as they are the natural
variables of $G$, and reduces to

Eq. ‑ $0 = - {N_{A}d\mu}_{A} - {N_{B}d\mu}_{B} - {N_{C}d\mu}_{C}$

A phase can thus be represented by a surface in the three-dimensional
space of $\mu_{A}$, $\mu_{B}$, and $\mu_{C}$ at constant $T$ and $P$
with similar geometric appearance as .

From , Gibbs energy of a ternary solution is written as

Eq. ‑
$G_{m} = x_{A}_{\ }^{0}G_{A} + x_{B}_{\ }^{0}G_{B} + x_{C}_{\ }^{0}G_{C} + RT\left( x_{A}l{nx}_{A} + x_{B}l{nx}_{B} + x_{C}l{nx}_{C} \right) +_{\ }^{E}G_{m}$

When the mole fraction of one component approaches zero,
$_{\ }^{E}G_{m}$ reduces to the excess Gibbs energy of mixing of the
binary systems of the remaining two components, represented by .
However, for a given composition of a ternary solution, there is no
unique way to assign the contributions from $_{\ }^{E}G_{m}$ of each
binary to $_{\ }^{E}G_{m}$ of the ternary solution because
$_{\ }^{E}G_{m}$ of the ternary solution contains information of both
binary and ternary interactions. A variety of models is available in the
literature (see \[1\]). One intuitive approach would be to use the same
formula as that in the binary system, i.e. , with the mole fractions
substituted by the values in the ternary system, and $_{\ }^{E}G_{m}$ of
a ternary solution may thus be defined as the following by including the
ternary interaction involving all three components,

Eq. ‑
$_{\ }^{E}G_{m} = x_{A}x_{B}L_{AB} + x_{A}x_{C}L_{AC} + x_{B}x_{C}L_{BC} + x_{A}x_{B}x_{C}L_{ABC}$

The chemical potential of a component is represented by . When all
interaction parameters in are constant, i.e. a ternary regular solution,
the chemical potential of component $A$ can be derived as

Eq. ‑
$\mu_{A} = G_{A} =_{\ }^{0}G_{A} + RTlnx_{A} + x_{B}L_{AB} + x_{C}L_{AC} -_{\ }^{E}G_{m} =_{\ }^{0}G_{A} + RTlnx_{A} + x_{B}{\left( 1 - x_{A} \right)L}_{AB} + x_{C}\left( 1 - x_{A} \right)L_{AC} - x_{B}x_{C}L_{BC} + x_{B}x_{C}\left( 1 - 2x_{A} \right)L_{ABC} =_{\ }^{0}G_{A} + RTlnx_{B} + x_{B}^{2}L_{AB} + x_{C}^{2}L_{AC} + x_{B}x_{C}\left( L_{AB} + L_{AC} - L_{BC} \right) + x_{B}x_{C}\left( 1 - 2x_{A} \right)L_{ABC}$

Similar equations can be derived for component $B$ and C with
$L_{AB} = L_{BA}$, $L_{AC} = L_{CA}$, and $L_{BC} = L_{CB}$. A schematic
molar Gibbs energy diagram at constant temperature and pressure is shown
in with all three binary systems having $L_{ij} < 0$ of similar values.

Figure ‑: Schematic ternary molar Gibbs energy diagram as a function of
compositions for given temperature and pressure

To evaluate the stability of a ternary solution, one needs to calculate
the elements in the determinant shown in . Using the mole of component
$C$ as the independent molar quantity, the limit of stability is
expresses as

Eq. ‑ $G_{AA}G_{BB} - G_{AB}G_{BA} = 0$

As an example, $G_{AA}$ is shown in the following equation, which must
be positive for the solution to be stable

Eq. ‑
${N\left( \frac{\partial\mu_{A}}{\partial N_{A}} \right)}_{T,P,N_{B},N_{C}} = NG_{AA} = \frac{RT\left( 1 - x_{A} \right)}{x_{A}} - 2x_{B}^{2}L_{AB} - 2x_{C}^{2}L_{AC} - 2x_{B}x_{C}\left( L_{AB} + L_{AC} - L_{BC} \right) - 2x_{B}x_{C}\left( 2 - 3x_{A} \right)L_{ABC}$

It is evident that any instability in binary systems with positive
interaction parameters extends into the ternary system. It can also be
seen that even if all binary interaction parameters are negative, i.e.
no instability in the binary systems, it is possible that becomes
negative for some combinations of the binary interaction parameters such
that $\mathrm{\Delta}L = L_{AB} + L_{AC} - L_{BC}$ becomes very positive
and overshadows the contributions due to $L_{AB}$ and $L_{AC}$, i.e.
$L_{BC}$ is more negative than $L_{AB}$ and $L_{AC}$ combined. In an
extreme case with $L_{AB} = L_{AC} = L_{ABC} = 0$ and $L_{BC} < 0$, i.e.
ideal solutions for the $A - B$ and $A - C$ binary systems, stable
solution in the $B - C$ binary system, and no additional ternary
interaction, reduces to

Eq. ‑
$N\left( \frac{\partial\mu_{A}}{\partial N_{A}} \right)_{T,P,N_{B},N_{C}} = \frac{RT\left( 1 - x_{A} \right)}{x_{A}} + 2x_{B}x_{C}L_{BC}$

Setting
$\left( \frac{\partial\mu_{A}}{\partial N_{A}} \right)_{T,P,N_{B},N_{C}} = 0$,
one obtains

Eq. ‑
$- \frac{RT}{2L_{BC}} = \frac{x_{A}x_{B}x_{C}}{1 - x_{A}} = \frac{{\left( 1 - x_{B} - x_{C} \right)x}_{B}x_{C}}{x_{B} + x_{C}}$

With $- \frac{RT}{2L_{BC}}$ being positive due to $L_{BC} < 0$, there is
a parabola-shaped composition area in which the solution is unstable at
constant temperature and pressure. This is reasonable because the system
tends to maximize the number of B-C bonds due to its lower energy, which
competes with the entropy of mixing among the three elements and results
in segregation of B-C bonds, thus miscibility gap at low temperatures.

To evaluate the ternary consolute point, the second derivatives for
component A and B are obtained as

Eq. ‑
${N\left( \frac{\partial_{\ }^{2}\mu_{A}}{\partial N_{A}^{2}} \right)}_{T,P,N_{B},N_{C}} = \frac{RT\left( 1 - x_{A} \right)}{x_{A}^{2}} + 4x_{B}^{2}L_{AB} + 4x_{C}^{2}L_{AC} + 4x_{B}x_{C}\left( L_{AB} + L_{AC} - L_{BC} \right) + 2x_{B}x_{C}\left( 7 - 9x_{A} \right)L_{ABC} = 0$

Eq. ‑
${N\left( \frac{\partial_{\ }^{2}\mu_{B}}{\partial N_{B}^{2}} \right)}_{T,P,N_{A},N_{C}} = \frac{RT\left( 1 - x_{B} \right)}{x_{B}^{2}} + 4x_{A}^{2}L_{AB} + 4x_{C}^{2}L_{BC} + 4x_{A}x_{C}\left( L_{AB} + L_{BC} - L_{AC} \right) + 2x_{A}x_{C}\left( 7 - 9x_{B} \right)L_{ABC} = 0$

The consolute point can then be obtained using , and .

It is observed in that $\left( 1 - 2x_{A} \right)L_{ABC} = 0$ at
$x_{A} = 0.5$, i.e. the ternary interaction parameter does not
contribute to the chemical potential of $A$. It is also observed in that
the contribution from the ternary interaction parameter changes sign at
$x_{i} = 2/3$ due to $\left( 2 - 3x_{A} \right)L_{ABC} = 0$.
